### Introduction

Welcome! This is the coding, visualization and Modeling part of the part of the project!

Basic approach here is to get the data needed from foursquare, visualize it, check if the data is sufficient and maybe repeat some of the process, then we try different clustering algorithms and evaluate those, to hopefully come to a conclusion!

Important comment: We will be working with the k-means algorithm a lot. As this algorithm is based on random starting points at the start of the algorithm, numbers and results might be different if the code is re-run from when I ran them originally!

Lets dive into it:

In [2]:
# getting libraries needed
import requests
import pandas as pd 
import numpy as np 
import time
    
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Success! Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

lets start off by visualizing the trade area of the business:

In [3]:
# starting a Folium map around the area:
map_ostfriesland = folium.Map(location=[53.23, 7.47], zoom_start=9)

# starting a list of points that mark the corner of the area
points = [[53.24, 7.21], [53.03, 7.21], [53.03, 8.57], [53.83, 8.57], [53.83, 6.07], [53.24, 7.21]]

# draw points
for each in points:  
    map_ostfriesland.add_child(folium.CircleMarker(location=each,
    fill='true',
    radius = 6,
    popup= str(each),
    fill_color='blue',
    color = 'clear',
    fill_opacity=1))

# add lines between the points
folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(map_ostfriesland)    

# lets draw the map!
map_ostfriesland

that looks like a challenge to get all the venues in that trade area, as its not circular or rectangular. As forsquare is working with a radius, and maximum 50 results per search, lets map several circles within the trade area to keep the result lists below 50:

In [4]:
# starting a Folium map around the area:
map_ostfriesland2 = folium.Map(location=[53.23, 7.47], zoom_start=9)

# starting a list of points that mark the corner of the area
points = [[53.24, 7.21], [53.03, 7.21], [53.03, 8.57], [53.83, 8.57], [53.83, 6.07], [53.24, 7.21]]

# draw points
for each in points:  
    map_ostfriesland2.add_child(folium.CircleMarker(location=each,
    fill='true',
    radius = 6,
    popup= 'Hi',
    fill_color='blue',
    color = 'clear',
                                                    
    fill_opacity=1))

# add lines between the points
folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(map_ostfriesland2)   

#creating empty list and variable for the loop
lon_factor=0
searchpoints=[]

# adding circles
for i in range(6):
    lon_factor+=0.33
    lat_factor=0
    for j in range(4):
        searchpoints.append([53.69+lat_factor, 6.45+lon_factor])
        map_ostfriesland2.add_child(folium.Circle(location=[53.69+lat_factor, 6.45+lon_factor], radius=15000,
                    line_color='red', fill_opacity = 0.3, fill='true',
                    fill_color='yellow'))
        lat_factor-=0.19
        

# lets draw the map!
map_ostfriesland2

24 circles with a radius of 15km seems to get all the area, but also a lot more - we will have to do some cleaning on the data!

Lets get foursquare set up with the needed parameters and start our search:

In [5]:
# setting up the Foursquare API paramenters:

#login credentials
CLIENT_ID = '4PNCQ5FGSPM5OJ1L0AYGTZWTAIOYAMSFTVRS0EFZQ02JU3VL' # your Foursquare ID
CLIENT_SECRET = 'JNKEW3N0WX1LEXFX0BJHIZKEBD5PXE30NNGYANZRFVYDTYV5' # your Foursquare Secret
VERSION = '20200620'

# lets start off by looking at restaurants, and a radius of 15km around each ccordinate 
search_query = 'Restaurant'
radius = 15000

#create empty pandas dataframe
df = pd.DataFrame()

# with a loop, we do one search for each of the 24 coordinate pairs, printing the coordinates and shapes to see how many restaurants we find:
for ll in searchpoints:
    print(ll)
    # calling the API
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, ll[0], ll[1], VERSION, search_query, radius)
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    venues = results['response']['venues']
    # tranform venues into a dataframe
    df_temp = json_normalize(venues)
    print(df_temp.shape)
    # add the current local search to the overall result list
    df = df.append(df_temp, sort=True)
    # sleep to not spam the API
    time.sleep(1)

[53.69, 6.78]
(14, 16)
[53.5, 6.78]
(20, 16)
[53.309999999999995, 6.78]
(30, 17)
[53.12, 6.78]
(30, 18)
[53.69, 7.11]
(30, 17)
[53.5, 7.11]
(29, 17)
[53.309999999999995, 7.11]
(29, 18)
[53.12, 7.11]
(29, 16)
[53.69, 7.44]
(21, 19)
[53.5, 7.44]
(10, 16)
[53.309999999999995, 7.44]
(28, 18)
[53.12, 7.44]
(30, 18)
[53.69, 7.7700000000000005]
(24, 19)
[53.5, 7.7700000000000005]
(12, 16)
[53.309999999999995, 7.7700000000000005]
(6, 16)
[53.12, 7.7700000000000005]
(9, 16)
[53.69, 8.1]
(22, 18)
[53.5, 8.1]
(30, 19)
[53.309999999999995, 8.1]
(26, 17)
[53.12, 8.1]
(30, 17)
[53.69, 8.43]
(8, 18)
[53.5, 8.43]
(30, 18)
[53.309999999999995, 8.43]
(13, 16)
[53.12, 8.43]
(30, 18)


In [7]:
# we repeat this process with a search query for 'Cafe' as well:
search_query = 'Cafe'
for ll in searchpoints:
    print(ll)
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, ll[0], ll[1], VERSION, search_query, radius)
    results = requests.get(url).json()
    venues = results['response']['venues']
    df_temp = json_normalize(venues)
    print(df_temp.shape)
    df = df.append(df_temp, sort=True)
    time.sleep(1)

[53.69, 6.78]
(21, 17)
[53.5, 6.78]
(28, 18)
[53.309999999999995, 6.78]
(30, 18)
[53.12, 6.78]
(30, 17)
[53.69, 7.11]
(21, 17)
[53.5, 7.11]
(30, 18)
[53.309999999999995, 7.11]
(30, 19)
[53.12, 7.11]
(30, 16)
[53.69, 7.44]
(19, 18)
[53.5, 7.44]
(6, 16)
[53.309999999999995, 7.44]
(30, 18)
[53.12, 7.44]
(30, 17)
[53.69, 7.7700000000000005]
(30, 18)
[53.5, 7.7700000000000005]
(11, 17)
[53.309999999999995, 7.7700000000000005]
(6, 17)
[53.12, 7.7700000000000005]
(11, 16)
[53.69, 8.1]
(19, 17)
[53.5, 8.1]
(30, 18)
[53.309999999999995, 8.1]
(30, 18)
[53.12, 8.1]
(30, 18)
[53.69, 8.43]
(7, 17)
[53.5, 8.43]
(30, 19)
[53.309999999999995, 8.43]
(13, 18)
[53.12, 8.43]
(30, 18)


In [8]:
#we copy the df, so we have a clean copy of the original dataframe so we can play around with the data without having to recall the API needlessly in case we need the original data.
df_clean = df.copy()

#reset the index:
df_clean.reset_index(inplace=True)
df_clean


,index,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,0,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,55e2ffd5498ed6652fc8eb5d,NaN,DE,NaN,Deutschland,NaN,12326,[Deutschland],"[{'label': 'display', 'lat': 53.591618, 'lng':...",53.591618,6.694292,NaN,NaN,NaN,Restaurant Café Geflügelhof,v-1592752936,NaN
1,1,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,51f550cf498e29a9dc99986f,NaN,DE,NaN,Deutschland,NaN,13162,[Deutschland],"[{'label': 'display', 'lat': 53.59354782104492...",53.593548,6.664623,NaN,NaN,NaN,Restaurant Zum Kaap,v-1592752936,NaN
2,2,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,50521591e4b0d453755ceed3,NaN,DE,NaN,Deutschland,NaN,13699,[Deutschland],"[{'label': 'display', 'lat': 53.58814211906586...",53.588142,6.663512,NaN,NaN,NaN,Restaurant Seehund,v-1592752936,NaN
3,3,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,4ebc17d3be7b329d90ba532d,NaN,DE,Borkum,Deutschland,NaN,13589,"[Borkum, Deutschland]","[{'label': 'display', 'lat': 53.589037, 'lng':...",53.589037,6.664241,NaN,NaN,Niedersachsen,Restaurant Austernfischer,v-1592752936,NaN
4,4,"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",False,4ffed466e4b002ba9a0b971e,NaN,DE,NaN,Deutschland,NaN,13064,[Deutschland],"[{'label': 'display', 'lat': 53.58511322759146...",53.585113,6.691201,NaN,NaN,NaN,Restaurant Zum Yachthafen,v-1592752936,NaN
5,5,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,513b9e25e4b02424670f5926,Jann-Berghaus-Str.63,DE,Borkum,Deutschland,NaN,13628,"[Jann-Berghaus-Str.63, 26757 Borkum, Deutschland]","[{'label': 'display', 'lat': 53.59028186130264...",53.590282,6.660200,NaN,26757,Niedersachsen,Restaurant Palée,v-1592752936,NaN
6,6,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,599b0df4419a9e32cdaef1b0,Süderstrasse 69,DE,Borkum,Deutschland,NaN,14232,"[Süderstrasse 69, 26757 Borkum, Deutschland]","[{'label': 'display', 'lat': 53.58098, 'lng': ...",53.580980,6.667349,NaN,26757,Niedersachsen,Restaurant Südhauk,v-1592752936,NaN
7,7,"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",False,4e69ecca1f6e1e74e3fb228b,NaN,DE,Borkum,Deutschland,NaN,10197,"[Borkum, Deutschland]","[{'label': 'display', 'lat': 53.604226133449, ...",53.604226,6.725730,NaN,NaN,Niedersachsen,Café Restaurant Ostland,v-1592752936,NaN
8,8,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,4ffee479e4b0092e51a67fe2,Roelof-Gerritz-Meyer-Str. 10,DE,Borkum,Deutschland,NaN,13430,"[Roelof-Gerritz-Meyer-Str. 10, 26757 Borkum, D...","[{'label': 'display', 'lat': 53.58788336997905...",53.587883,6.671627,NaN,26757,Niedersachsen,Cafe Restaurant Alt Borkum,v-1592752936,NaN
9,9,"[{'id': '4bf58dd8d48988d155941735', 'name': 'G...",False,555502a8498ea54200dd3155,Hindenburgstr. 97,DE,Borkum,Deutschland,NaN,12565,"[Hindenburgstr. 97, 26757 Borkum, Deutschland]","[{'label': 'display', 'lat': 53.5956572089344,...",53.595657,6.675450,NaN,26757,Niedersachsen,Café Restaurant Bar Alibi,v-1592752936,NaN


In [9]:
#drop all columns that are not needed for the problem we are trying to solve
df_clean2 = df_clean.loc[:, ['name', 'id', 'categories', 'location.lat', 'location.lng']]

In [10]:
# lets do a quick helper function to extract the categories out of the list currently saved there
def get_category_type(row):
    categories_list = row['categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [11]:
# and lets call the funtion to get a clean category
df_clean2['categories'] = df_clean2.apply(get_category_type, axis=1)

df_clean2.head()

,name,id,categories,location.lat,location.lng
0,Restaurant Café Geflügelhof,55e2ffd5498ed6652fc8eb5d,Café,53.591618,6.694292
1,Restaurant Zum Kaap,51f550cf498e29a9dc99986f,Seafood Restaurant,53.593548,6.664623
2,Restaurant Seehund,50521591e4b0d453755ceed3,Seafood Restaurant,53.588142,6.663512
3,Restaurant Austernfischer,4ebc17d3be7b329d90ba532d,Seafood Restaurant,53.589037,6.664241
4,Restaurant Zum Yachthafen,4ffed466e4b002ba9a0b971e,German Restaurant,53.585113,6.691201


In [12]:
# and drop duplicates and reset index
df_clean2.drop_duplicates(subset=['id'], inplace=True)
df_clean2.reset_index(inplace=True)
df_clean2 = df_clean2.loc[:, ['name', 'id', 'categories', 'location.lat', 'location.lng']]
df_clean2

,name,id,categories,location.lat,location.lng
0,Restaurant Café Geflügelhof,55e2ffd5498ed6652fc8eb5d,Café,53.591618,6.694292
1,Restaurant Zum Kaap,51f550cf498e29a9dc99986f,Seafood Restaurant,53.593548,6.664623
2,Restaurant Seehund,50521591e4b0d453755ceed3,Seafood Restaurant,53.588142,6.663512
3,Restaurant Austernfischer,4ebc17d3be7b329d90ba532d,Seafood Restaurant,53.589037,6.664241
4,Restaurant Zum Yachthafen,4ffed466e4b002ba9a0b971e,German Restaurant,53.585113,6.691201
5,Restaurant Palée,513b9e25e4b02424670f5926,Restaurant,53.590282,6.660200
6,Restaurant Südhauk,599b0df4419a9e32cdaef1b0,Restaurant,53.580980,6.667349
7,Café Restaurant Ostland,4e69ecca1f6e1e74e3fb228b,German Restaurant,53.604226,6.725730
8,Cafe Restaurant Alt Borkum,4ffee479e4b0092e51a67fe2,Seafood Restaurant,53.587883,6.671627
9,Café Restaurant Bar Alibi,555502a8498ea54200dd3155,Gastropub,53.595657,6.675450


In [32]:
#now we need to drop all data that wasnt in the original trade area borders:

#remove all values south of the southern border
df_clean3 = df_clean2[df_clean2['location.lat'] > 53.03]
print(df_clean3.shape)

#remove all values east of the border
df_clean3 = df_clean3[df_clean3['location.lng'] < 8.49]
print(df_clean3.shape)

#remove all values west of the border
df_clean3.rename(columns={"location.lat": "lat", "location.lng": "lng"},inplace=True)
df_clean3.drop(df_clean3[(df_clean3.lng < 7.21) & (df_clean3.lat < 53.32)].index, inplace=True)
df_clean3.drop(df_clean3[(df_clean3.lng < 7) & (df_clean3.lat < 53.53)].index, inplace=True)
print(df_clean3.shape)



(706, 5)
(635, 5)
(466, 5)


In [33]:
# lets check what categories we have, to exclude any asian restaurants:
df_clean3.groupby(['categories']).count()

,name,id,lat,lng
categories,,,,
American Restaurant,1,1,1,1
Asian Restaurant,4,4,4,4
BBQ Joint,3,3,3,3
Bagel Shop,2,2,2,2
Bakery,9,9,9,9
Bar,6,6,6,6
Bistro,2,2,2,2
Breakfast Spot,2,2,2,2
Burger Joint,2,2,2,2


In [35]:
# we can see fitting categrories: Asian Restaurant, Dumpling Restaurant, Chinese Restaurant, Mongolian Restaurant,
#df_clean4 = df_clean3[df_clean3['categories'] != ('Asian Restaurant' or 'Dumpling Restaurant' or 'Chinese Restaurant' or 'Mongolian Restaurant')]
df_clean3.drop(df_clean3[(df_clean3.categories == 'Asian Restaurant')].index, inplace=True)
df_clean3.drop(df_clean3[(df_clean3.categories == 'Chinese Restaurant')].index, inplace=True)
df_clean3.drop(df_clean3[(df_clean3.categories == 'Dumpling Restaurant')].index, inplace=True)
df_clean3.drop(df_clean3[(df_clean3.categories == 'Mongolian Restaurant')].index, inplace=True)
df_clean3.reset_index(inplace=True)
df_clean3 = df_clean3.loc[:, ['name', 'id', 'categories', 'lat', 'lng']]
df_cleaned = df_clean3.copy()
df_cleaned

,name,id,categories,lat,lng
0,Restaurant Café Geflügelhof,55e2ffd5498ed6652fc8eb5d,Café,53.591618,6.694292
1,Restaurant Zum Kaap,51f550cf498e29a9dc99986f,Seafood Restaurant,53.593548,6.664623
2,Restaurant Seehund,50521591e4b0d453755ceed3,Seafood Restaurant,53.588142,6.663512
3,Restaurant Austernfischer,4ebc17d3be7b329d90ba532d,Seafood Restaurant,53.589037,6.664241
4,Restaurant Zum Yachthafen,4ffed466e4b002ba9a0b971e,German Restaurant,53.585113,6.691201
5,Restaurant Palée,513b9e25e4b02424670f5926,Restaurant,53.590282,6.660200
6,Restaurant Südhauk,599b0df4419a9e32cdaef1b0,Restaurant,53.580980,6.667349
7,Café Restaurant Ostland,4e69ecca1f6e1e74e3fb228b,German Restaurant,53.604226,6.725730
8,Cafe Restaurant Alt Borkum,4ffee479e4b0092e51a67fe2,Seafood Restaurant,53.587883,6.671627
9,Café Restaurant Bar Alibi,555502a8498ea54200dd3155,Gastropub,53.595657,6.675450


Now we finally have it! Our clean dataframe of 451 restaurants that we can use for further analysis! Lets draw a map first, to visualize the data:

In [218]:
# starting a Folium map around the area:
map_rest = folium.Map(location=[53.23, 7.47], zoom_start=9)

# starting a list of points that mark the corner of the area
map_rest = [[53.53, 6.5], [53.24, 6.5], [53.24, 7.0], [53.32, 7.21]]

# draw points
for each in points:  
    map_rest.add_child(folium.CircleMarker(location=each,
    fill='true',
    radius = 6,
    popup= str(each),
    fill_color='blue',
    color = 'clear',
    fill_opacity=1))

# add lines between the points
folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(map_rest)    

# lets draw the map!
map_rest

AttributeError: 'list' object has no attribute 'add_child'

In [37]:
venues_map = folium.Map(location=[53.23, 7.47], zoom_start=9)


# add popular spots to the map as blue circle markers
for lat, lng in zip(df_cleaned.lat, df_cleaned.lng):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

Now that we have all the data, lets look at finding a model that does what the original question was - how can we get an efficient and fair solution in distibuting the trade areas?

We will use the power of machine learning here, namely three algorithms of unsupervised learning that are the most common clustering methods: k-means clustering, hierachical clustering, and DBSCAN clustering.

In [195]:
# we start with k-means clustering, in this case 5 clusters
from sklearn.cluster import KMeans

# we copy the dataframe
df_kmeans = df_cleaned.copy()

# arrange latitude and longitude in the correct format
df_latlng = df_kmeans[['lat', 'lng']]

# run k-means clustering with 5 clusters depending on latitude and longitude
kmeans = KMeans(n_clusters=5).fit(df_latlng)

#insert Clusters into the df
df_kmeans.insert(0, 'Cluster Labels', kmeans.labels_)

#lets see what this did:
df_kmeans.head(5)


,Cluster Labels,name,id,categories,lat,lng
0,2,Restaurant Café Geflügelhof,55e2ffd5498ed6652fc8eb5d,Café,53.591618,6.694292
1,2,Restaurant Zum Kaap,51f550cf498e29a9dc99986f,Seafood Restaurant,53.593548,6.664623
2,2,Restaurant Seehund,50521591e4b0d453755ceed3,Seafood Restaurant,53.588142,6.663512
3,2,Restaurant Austernfischer,4ebc17d3be7b329d90ba532d,Seafood Restaurant,53.589037,6.664241
4,2,Restaurant Zum Yachthafen,4ffed466e4b002ba9a0b971e,German Restaurant,53.585113,6.691201


lets put this into our same map with different colors for the clusters:

In [196]:
# same map, this time different colors based on cluster labels:
venues_kmeans = folium.Map(location=[53.23, 7.47], zoom_start=9)

# add markers to map
for lat, lng, cluster in zip(df_kmeans['lat'], df_kmeans['lng'], df_kmeans['Cluster Labels']):
    if cluster == 0:
        color = 'Red'
    elif cluster == 1:
        color = 'Blue'
    elif cluster == 2:
        color = 'Green'
    elif cluster == 3:
        color = 'Black' 
    elif cluster == 4:
        color = 'Orange'
    elif cluster == 5:
        color = 'pink' 
    elif cluster == 6:
        color = 'white' 
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        parse_html=False).add_to(venues_kmeans)  
    
venues_kmeans

In [197]:
df_kmeans.groupby(['Cluster Labels']).count()

,name,id,categories,lat,lng
Cluster Labels,,,,,
0,67,67,63,67,67
1,109,109,105,109,109
2,25,25,25,25,25
3,137,137,131,137,137
4,113,113,110,113,113


our first cluster! At a first clange we can see that the island on the far left (Borkum) is rather far away from the rest, so the algorithm gives it its own cluster with only 25 restaurants. The rest seems understandable and logical, with some variance in it (67 min to 137 max), but 3 of the 5 rather close. Lets see the next algorithm:

In [198]:
#now Hierachical Clustering!
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler

# we copy the dataframe
df_hier = df_cleaned.copy()

# arrange latitude and longitude in the correct format
df_latlng = df_hier[['lat', 'lng']]

# we turn the values in a numpy array and normalize them:
x = df_latlng.values
min_max_scaler = MinMaxScaler()
feature_mtx = min_max_scaler.fit_transform(x)
feature_mtx [0:5]

# run agglomaritive hiercharical clustering with 5 clusters depending on latitude and longitude
agglom = AgglomerativeClustering(n_clusters = 5)
agglom.fit(feature_mtx)

#insert Clusters into the df
df_hier.insert(0, 'Cluster Labels', agglom.labels_)

#lets see what this did:
df_hier.head()


,Cluster Labels,name,id,categories,lat,lng
0,0,Restaurant Café Geflügelhof,55e2ffd5498ed6652fc8eb5d,Café,53.591618,6.694292
1,0,Restaurant Zum Kaap,51f550cf498e29a9dc99986f,Seafood Restaurant,53.593548,6.664623
2,0,Restaurant Seehund,50521591e4b0d453755ceed3,Seafood Restaurant,53.588142,6.663512
3,0,Restaurant Austernfischer,4ebc17d3be7b329d90ba532d,Seafood Restaurant,53.589037,6.664241
4,0,Restaurant Zum Yachthafen,4ffed466e4b002ba9a0b971e,German Restaurant,53.585113,6.691201


In [199]:
# same map, this time different colors based on cluster labels:
venues_hier = folium.Map(location=[53.23, 7.47], zoom_start=9)

for lat, lng, cluster in zip(df_hier['lat'], df_hier['lng'], df_hier['Cluster Labels']):
    if cluster == 0:
        color = 'Red'
    elif cluster == 1:
        color = 'Blue'
    elif cluster == 2:
        color = 'Green'
    elif cluster == 3:
        color = 'Black' 
    elif cluster == 4:
        color = 'Orange'
    elif cluster == 5:
        color = 'pink' 
    elif cluster == 6:
        color = 'white' 
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        parse_html=False).add_to(venues_hier)  
    
venues_hier

In [200]:
df_hier.groupby(['Cluster Labels']).count()

,name,id,categories,lat,lng
Cluster Labels,,,,,
0,164,164,158,164,164
1,88,88,86,88,88
2,76,76,74,76,76
3,64,64,60,64,64
4,59,59,56,59,59


Here we can see, out little Island-Problem is solved, however this algorithm brings almost half of the map together in the red cluster, with a total of 164 restaurants, leaving only 59-88 restaurants in the other clusters. Thats a big lean towards the red cluster. Lets see what our last algorithm brings:

In [131]:
from sklearn.cluster import DBSCAN

# we copy the dataframe
df_DBS = df_cleaned.copy()

# arrange latitude and longitude in the correct format
df_latlng = df_DBS[['lat', 'lng']]

# run agglomaritive hiercharical clustering with 5 clusters depending on latitude and longitude
epsilon = 0.05
minimumSamples = 6
DBS = DBSCAN(eps=epsilon, min_samples=minimumSamples).fit(df_latlng)

#insert Clusters into the df
df_DBS.insert(0, 'Cluster Labels', DBS.labels_)

#lets see what this did:
df_DBS.head(5)

,Cluster Labels,name,id,categories,lat,lng
0,0,Restaurant Café Geflügelhof,55e2ffd5498ed6652fc8eb5d,Café,53.591618,6.694292
1,0,Restaurant Zum Kaap,51f550cf498e29a9dc99986f,Seafood Restaurant,53.593548,6.664623
2,0,Restaurant Seehund,50521591e4b0d453755ceed3,Seafood Restaurant,53.588142,6.663512
3,0,Restaurant Austernfischer,4ebc17d3be7b329d90ba532d,Seafood Restaurant,53.589037,6.664241
4,0,Restaurant Zum Yachthafen,4ffed466e4b002ba9a0b971e,German Restaurant,53.585113,6.691201


In [132]:
import random
# same map, this time different colors based on cluster labels:
venues_DBS = folium.Map(location=[53.23, 7.47], zoom_start=9)

list_colors=['Red', 'Blue','Green','Black', 'Orange','pink','white', 'Red', 'Blue','Green','Black', 'Orange','pink','white','Red', 'Blue','Green','Black', 'Orange','pink','white','Red', 'Blue','Green','Black', 'Orange','pink','white','Red', 'Blue','Green','Black', 'Orange','pink','white','Red', 'Blue','Green','Black', 'Orange','pink','white', 'grey']
      
for lat, lng, cluster in zip(df_DBS['lat'], df_DBS['lng'], df_DBS['Cluster Labels']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=list_colors[cluster],
        fill=True,
        fill_color=list_colors[cluster],
        
        fill_opacity=0.9,
        parse_html=False).add_to(venues_DBS)
    
    
venues_DBS

In [133]:
df_DBS.groupby(['Cluster Labels']).count()

,name,id,categories,lat,lng
Cluster Labels,,,,,
-1,103,103,100,103,103
0,25,25,25,25,25
1,25,25,23,25,25
2,21,21,19,21,21
3,11,11,11,11,11
4,24,24,24,24,24
5,11,11,11,11,11
6,7,7,7,7,7
7,11,11,10,11,11


Maybe the DBSCAN algorithm wasnt the correct one from the beginning to answer the stakeholders question, as it is designed to mark outliers and not include them in their clusters - thats not what we want, we want to have all restaurants attributed to one sales area! However this map does have its insights, showing the bigger accumulation of restaurants, so Hot-Spots for our sales team.

To summarize the initial insights from the three algorithms: k-means is marking one island as a sperate cluster, otherwise seems a solid base to start further investigations. The hierarchical clustering produces clusters of very uneven size, so thats not so optimal for our goal so find fair cluster sizes. The DBSCAN does help with interesting insights, however it does not answer the original question for this project.

So, lets continue to see if we can get the k-means algorithm to get us closer to an answer! Lets start off by playing with the cluster size (maybe 4 or 6 cluster might help, its within the scope of the stakeholders question):

In [203]:
# we copy the code from above, but change to 4 clusters only:
df_kmeans2 = df_cleaned.copy()
df_latlng2 = df_kmeans2[['lat', 'lng']]
kmeans2 = KMeans(n_clusters=4).fit(df_latlng2)
df_kmeans2.insert(0, 'Cluster Labels', kmeans2.labels_)
venues_kmeans2 = folium.Map(location=[53.23, 7.47], zoom_start=9)
for lat, lng, cluster in zip(df_kmeans2['lat'], df_kmeans2['lng'], df_kmeans2['Cluster Labels']):
    if cluster == 0:
        color = 'Red'
    elif cluster == 1:
        color = 'Blue'
    elif cluster == 2:
        color = 'Green'
    elif cluster == 3:
        color = 'Black' 
    elif cluster == 4:
        color = 'Orange'
    elif cluster == 5:
        color = 'pink' 
    elif cluster == 6:
        color = 'white' 
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        parse_html=False).add_to(venues_kmeans2)    
venues_kmeans2

In [204]:
# lets see how many restaurants we have per cluster
df_kmeans2.groupby(['Cluster Labels']).count()

,name,id,categories,lat,lng
Cluster Labels,,,,,
0,103,103,98,103,103
1,109,109,104,109,109
2,117,117,113,117,117
3,122,122,119,122,122


Thats a very even distribution, and gets rid of the "island-problem". Minimum 103 restaurants, maximum 122. That sounds workable when looking at fairness and efficiency. We have a possible solution here. But what happens when we have 6 clusters? Lets see:

In [207]:
# we copy the code from above, but change to 6 clusters:
df_kmeans3 = df_cleaned.copy()
df_latlng3 = df_kmeans3[['lat', 'lng']]
kmeans3 = KMeans(n_clusters=6).fit(df_latlng3)
df_kmeans3.insert(0, 'Cluster Labels', kmeans3.labels_)
venues_kmeans3 = folium.Map(location=[53.23, 7.47], zoom_start=9)
for lat, lng, cluster in zip(df_kmeans3['lat'], df_kmeans3['lng'], df_kmeans3['Cluster Labels']):
    if cluster == 0:
        color = 'Red'
    elif cluster == 1:
        color = 'Blue'
    elif cluster == 2:
        color = 'Green'
    elif cluster == 3:
        color = 'Black' 
    elif cluster == 4:
        color = 'Orange'
    elif cluster == 5:
        color = 'pink' 
    elif cluster == 6:
        color = 'white' 
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        parse_html=False).add_to(venues_kmeans3)    
venues_kmeans3

In [208]:
df_kmeans3.groupby(['Cluster Labels']).count()

,name,id,categories,lat,lng
Cluster Labels,,,,,
0,69,69,67,69,69
1,25,25,25,25,25
2,93,93,90,93,93
3,102,102,98,102,102
4,84,84,81,84,84
5,78,78,73,78,78


That is offhand not a workable solution: it maintains the "island-problem", and min-max apart from the island is 69-102, so also not very fair. 6 Trade areas doesnt seem to be the solution.

But if we look beyond the numbers and algorithms, the trip to the island seems to a two-day sales trip - a trip there would anyways not be combinable with other visits on the mainland (ferry is about 3 hours). Maybe we can look at assigning the island to another trade area / cluster, independent of the distance to the trade area? If we add the 25 restaurants to the second smallest cluster (69), we would be at 94, our min-max would be 78-102, which seems fair, and we would be back at our original (optimal) trade areas! OK, lets assign cluster label 1 to cluster 0:

In [215]:
# copy df
df_recluster = df_kmeans3.copy()

#replace 1's with 0's
df_recluster['Cluster Labels'].replace(to_replace=1,value=0, inplace=True)
df_recluster.groupby(['Cluster Labels']).count()


,name,id,categories,lat,lng
Cluster Labels,,,,,
0,94,94,92,94,94
2,93,93,90,93,93
3,102,102,98,102,102
4,84,84,81,84,84
5,78,78,73,78,78


In [219]:
venues_kmeans4 = folium.Map(location=[53.23, 7.47], zoom_start=9)
for lat, lng, cluster in zip(df_recluster['lat'], df_recluster['lng'], df_recluster['Cluster Labels']):
    if cluster == 0:
        color = 'Red'
    elif cluster == 1:
        color = 'pink'
    elif cluster == 2:
        color = 'Green'
    elif cluster == 3:
        color = 'Black' 
    elif cluster == 4:
        color = 'Orange'
    elif cluster == 5:
        color = 'blue' 
    elif cluster == 6:
        color = 'white' 
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.9,
        parse_html=False).add_to(venues_kmeans4)    
venues_kmeans4

And here we have it! An alternative solution with 5 clusters / sales regions we can recommend to the stakeholders.

That means have 2 possible solutions within the parameters set, one with 4 trade areas and one with 5 trade areas.

Let us present our findings now!